## Лабораторная работа №3

In [32]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [59]:
# class MLPptorch(nn.Module):
#     def __init__(self,
#                  in_size,
#                  first_hidden_size,
#                  second_hidden_size,
#                  third_hidden_size,
#                  out_size):
#         nn.Module.__init__(self)
#         self.layers = nn.Sequential(nn.Linear(in_size, first_hidden_size),
#                                     nn.Sigmoid(),
#                                     nn.Linear(first_hidden_size, second_hidden_size),
#                                     nn.Sigmoid(),
#                                     nn.Linear(second_hidden_size, third_hidden_size),
#                                     nn.Sigmoid(),
#                                     nn.Linear(third_hidden_size, out_size),
#                                     nn.Sigmoid())

#     # прямой проход
#     def forward(self, x):
#         return self.layers(x)

class MLPptorch(nn.Module):
    def __init__(self,
                 in_size,
                 first_hidden_size,
                 second_hidden_size,
                 third_hidden_size,
                 out_size):
        nn.Module.__init__(self)
        self.layers = nn.Sequential(nn.Linear(in_size, first_hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(first_hidden_size, second_hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(second_hidden_size, third_hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(third_hidden_size, out_size),
                                    nn.Sigmoid())

    # прямой проход
    def forward(self, x):
        return self.layers(x)



In [60]:
# функция обучения
def train(x, y, num_iter):
    for i in range(0,num_iter):
        pred = net.forward(x)
        loss = lossFn(pred, y)
        loss.backward()
        optimizer.step()
        if i%(num_iter/10)==0:
           print('Ошибка на ' + str(i) + ' итерации: ', loss.item())
    return loss.item()


In [76]:
path = 'E:\\data.csv'
df = pd.read_csv(path)
df = df.iloc[np.random.permutation(len(df))]

X = df.iloc[0:100, 0:3].values
y = df.iloc[0:100, 4]
y = y.map({'Iris-setosa': 1, 'Iris-virginica': 2, 'Iris-versicolor':3}).values.reshape(-1,1)
Y = np.zeros((y.shape[0], np.unique(y).shape[0]))
for i in np.unique(y):
    Y[:,i-1] = np.where(y == i, 1, 0).reshape(1,-1)

X_test = df.iloc[100:150, 0:3].values
y = df.iloc[100:150, 4]
y = y.map({'Iris-setosa': 1, 'Iris-virginica': 2, 'Iris-versicolor':3}).values.reshape(-1,1)
Y_test = np.zeros((y.shape[0], np.unique(y).shape[0]))
for i in np.unique(y):
    Y_test[:,i-1] = np.where(y == i, 1, 0).reshape(1,-1)


inputSize = X.shape[1]  # количество входных сигналов равно количеству признаков задачи
first_hidden_size = 50  # задаем число нейронов скрытого слоя
second_hidden_size = 20  # задаем число нейронов скрытого слоя
third_hidden_size = 10  # задаем число нейронов скрытого слоя
outputSize = Y.shape[1] if len(Y.shape) else 1  # количество выходных сигналов равно количеству классов задачи

In [77]:
net = MLPptorch(inputSize,
                first_hidden_size,
                second_hidden_size,
                third_hidden_size,
                outputSize)
lossFn = nn.MSELoss()

optimizer = torch.optim.SGD(net.parameters(), lr=0.0003)

In [78]:
loss_ = train(torch.from_numpy(X.astype(np.float32)), 
              torch.from_numpy(Y.astype(np.float32)), 5000)

# for name, param in net.named_parameters():
#     print(name, param)

Ошибка на 0 итерации:  0.2624607980251312
Ошибка на 500 итерации:  0.10852322727441788
Ошибка на 1000 итерации:  0.048405177891254425
Ошибка на 1500 итерации:  0.056823182851076126
Ошибка на 2000 итерации:  0.036676499992609024
Ошибка на 2500 итерации:  0.03692644089460373
Ошибка на 3000 итерации:  0.11339106410741806
Ошибка на 3500 итерации:  0.2266666740179062
Ошибка на 4000 итерации:  0.2199999988079071
Ошибка на 4500 итерации:  0.3733333349227905


In [79]:
pred = net.forward(torch.from_numpy(X.astype(np.float32))).detach().numpy()
err = sum(abs((pred > 0.5) - Y))
print(err)   

pred = net.forward(torch.from_numpy(X_test.astype(np.float32))).detach().numpy()
err = sum(abs((pred > 0.5) - Y_test))
print(err)

[ 0. 33. 33.]
[ 0. 17. 17.]
